In [27]:
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img

In [75]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,BatchNormalization
from keras import backend as K
import keras

# dimensions of our images.
img_width, img_height = 300, 300

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 20025
nb_validation_samples = 4000
epochs = 20
batch_size = 64
opt = keras.optimizers.Adam(learning_rate=0.00001)

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(BatchNormalization())
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [76]:
path= "./PF_Data/"
train_datagen = ImageDataGenerator( rescale=1./255.0,
                                    rotation_range=20,
                                    zoom_range=0.15,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.15,
                                    horizontal_flip=True,
                                    fill_mode="nearest")
valid_datagen = ImageDataGenerator(rescale=1./255.0)
test_datagen = ImageDataGenerator(rescale=1./255.0)

In [77]:
train_generator = train_datagen.flow_from_directory(
    directory=path+"Train/",
    target_size=(300, 300),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42
)

Found 20025 images belonging to 2 classes.


In [78]:
valid_generator = valid_datagen.flow_from_directory(
    directory=path+"Valid/",
    target_size=(300, 300),
    color_mode="rgb",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42
)

Found 4000 images belonging to 2 classes.


In [79]:
train_generator.reset()
valid_generator.reset()

In [ ]:
       model.fit_generator( 
            train_generator,
            steps_per_epoch=nb_train_samples // batch_size,
            epochs=5,
            validation_data=valid_generator,
            validation_steps=nb_validation_samples // batch_size)

Epoch 1/5
 69/312 [=====>........................] - ETA: 26:14 - loss: 0.7696 - accuracy: 0.5284

In [46]:
model.evaluate(valid_generator,
steps=nb_validation_samples // batch_size)

62/62 [==============================] - 28s 452ms/step - loss: 0.1135 - accuracy: 0.9773


[0.11347514390945435, 0.977318525314331]

In [51]:
model.save_weights('my_model_weights.h5')

In [61]:
test_generator = test_datagen.flow_from_directory(
    directory=path+"test/",
    target_size=(300, 300),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 4 images belonging to 1 classes.


In [64]:
STEP_SIZE_TEST=4
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

4/4 [==============================] - 0s 16ms/step


In [65]:
pred

array([[0.05713782],
       [0.06823528],
       [0.33101892],
       [0.952749  ]], dtype=float32)

In [60]:
import numpy as np
np.round(pred,11)

array([[0.05713782],
       [0.06823528],
       [0.33101892]], dtype=float32)